In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from cellpose import models
from cellpose.io import imread
import glob
from pathlib import Path
from PIL import Image, ImageSequence
from tqdm import tqdm
import os
import os.path
from livecell_tracker import core
from livecell_tracker.core import datasets
from livecell_tracker.core.datasets import LiveCellImageDataset, SingleImageDataset
from skimage import measure
from livecell_tracker.core import SingleCellTrajectory, SingleCellStatic
import numpy as np
import os, json, cv2, random
import cv2

## SingleCellStatic: saving & loading 

In [ ]:
io_out_dir = Path("test_io_output")

In [ ]:
dataset_dir_path = Path(
    "../datasets/test_data_STAV-A549/DIC_data"
)

mask_dataset_path = Path("../datasets/test_data_STAV-A549/mask_data")


In [ ]:
mask_dataset = LiveCellImageDataset(mask_dataset_path, ext="png")

time2url = sorted(glob.glob(str((Path(dataset_dir_path) / Path("*_DIC.tif")))))
time2url = {i: path for i, path in enumerate(time2url)}
dic_dataset = LiveCellImageDataset(time2url=time2url, ext="tif")

In [ ]:
from skimage.measure import regionprops
from livecell_tracker.segment.utils import prep_scs_from_mask_dataset
single_cells = prep_scs_from_mask_dataset(mask_dataset, dic_dataset)

In [ ]:
single_cells[1].meta

In [ ]:
sc_json_list = SingleCellStatic.write_single_cells_json(single_cells, io_out_dir/"single_cells.json", dataset_dir=io_out_dir/"dataset", return_list=True)

In [ ]:
SingleCellStatic.write_single_cells_json(single_cells, io_out_dir/"single_cells.json", dataset_dir=io_out_dir/"dataset", return_list=False)

In [ ]:
img_dataset = single_cells[0].img_dataset

In [ ]:
loaded_scs = SingleCellStatic.load_single_cells_json(io_out_dir/"single_cells.json")

In [ ]:
loaded_scs[0]

In [ ]:
for sc in single_cells[:2]:
    print(sc.id)

In [ ]:
for sc in loaded_scs[:2]:
    print(sc.id)

In [ ]:
for sc in single_cells:
    for loaded_sc in loaded_scs:
        if sc.id == loaded_sc.id:
            # compare contour, contours are np.array
            assert np.allclose(sc.contour, loaded_sc.contour), f"the difference between sc.contour and loaded_sc.contour is {sc.contour - loaded_sc.contour}, ids are {sc.id} and {loaded_sc.id}"

## SingleCellTrajectory: saving & loading

In [ ]:
from typing import List
from livecell_tracker.track.sort_tracker_utils import (
    gen_SORT_detections_input_from_contours,
    update_traj_collection_by_SORT_tracker_detection,
    track_SORT_bbox_from_contours,
    track_SORT_bbox_from_scs
)


sct_collection = track_SORT_bbox_from_scs(single_cells, dic_dataset, mask_dataset=mask_dataset, max_age=1, min_hits=1)

In [ ]:
sct_collection.write_json(io_out_dir/"sct_collection.json")